In [129]:
import pandas as pd
from datetime import datetime
from slugify import slugify

In [130]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [131]:
df = pd.read_csv('csv/products.csv')
images = pd.read_csv('out/image.csv')

In [132]:
newDf = df[['Name', 'E-Tickets', 'Description']]

In [133]:
columns = ['id', 'createdDate', 'updatedDate', 'internalNote', 'creatorId', 'statusId', \
          'slug', 'shortDescription', 'alternativeName', 'mainImageId']

In [134]:
newDf = pd.concat([newDf, pd.DataFrame(columns = columns)])

In [135]:
newDf.rename(columns = {'Name':'name', 'E-Tickets' : 'onlineTicketUrl', \
                    'Description' : 'description'}, inplace = True)

In [136]:
newDf['id'] = newDf.index + 1
newDf['createdDate'] = dt_string
newDf['updatedDate'] = dt_string
newDf['internalNote'] = ''
newDf['statusId'] = 4
newDf['creatorId'] = 900

In [137]:
def set_slug(value):
    try:
        return slugify(str(value))
    except:
        return ''

newDf['slug'] = newDf['name'].apply(set_slug)

In [138]:
def get_short_description(value):
    return str(value)[:200]

newDf['shortDescription'] = newDf['description'].apply(get_short_description)

In [139]:
def get_alternative_name(value):
    temp = value.split(' ')
    return ', '.join(str(e) for e in temp)

newDf['alternativeName'] = newDf['name'].apply(get_alternative_name)

In [140]:
def get_main_image_id(value):
    try:
        tempName = str(value) + '_1'
        return images[:][images['name'].str.contains(tempName)]['id'].values[0]
    except:
        return ''
    
newDf['mainImageId'] = df['ID'].apply(get_main_image_id)

In [141]:
newDf.to_csv('out/products.csv', index=False, encoding='utf-8')